<a href="https://colab.research.google.com/github/anitaokoh/technical_articles_repo/blob/main/ML_in_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! kaggle datasets download -d yasserh/titanic-dataset
!unzip titanic-dataset.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/22.0k [00:00<?, ?B/s]
100% 22.0k/22.0k [00:00<00:00, 14.8MB/s]
Archive:  titanic-dataset.zip
  inflating: Titanic-Dataset.csv     


In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (92.1 kB/s)
Reading package li

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler,  StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from pyspark.mllib.evaluation import  MulticlassMetrics
spark = SparkSession.builder.master("local[4]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
df = spark.read.csv('*.csv', header=True, sep=",", inferSchema=True)
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [6]:
df = df.select("Sex", "Age", "Sibsp", "Fare", col("Survived").cast("double")).na.drop()
df.show()

+------+----+-----+-------+--------+
|   Sex| Age|Sibsp|   Fare|Survived|
+------+----+-----+-------+--------+
|  male|22.0|    1|   7.25|     0.0|
|female|38.0|    1|71.2833|     1.0|
|female|26.0|    0|  7.925|     1.0|
|female|35.0|    1|   53.1|     1.0|
|  male|35.0|    0|   8.05|     0.0|
|  male|54.0|    0|51.8625|     0.0|
|  male| 2.0|    3| 21.075|     0.0|
|female|27.0|    0|11.1333|     1.0|
|female|14.0|    1|30.0708|     1.0|
|female| 4.0|    1|   16.7|     1.0|
|female|58.0|    0|  26.55|     1.0|
|  male|20.0|    0|   8.05|     0.0|
|  male|39.0|    1| 31.275|     0.0|
|female|14.0|    0| 7.8542|     0.0|
|female|55.0|    0|   16.0|     1.0|
|  male| 2.0|    4| 29.125|     0.0|
|female|31.0|    1|   18.0|     0.0|
|  male|35.0|    0|   26.0|     0.0|
|  male|34.0|    0|   13.0|     1.0|
|female|15.0|    0| 8.0292|     1.0|
+------+----+-----+-------+--------+
only showing top 20 rows



In [7]:
df.printSchema()

root
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Sibsp: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Survived: double (nullable = true)



In [8]:
gender_indexer = StringIndexer(inputCol="Sex", outputCol="Gender")
columns = ['Gender', 'Age', 'Sibsp', 'Fare']
assembler = VectorAssembler(inputCols=columns, outputCol="v_features")
scaler = StandardScaler(inputCol="v_features", outputCol="features", withStd=True)
pipeline = Pipeline(stages=[gender_indexer, assembler, scaler])
data = pipeline.fit(df).transform(df)


In [9]:
data.show()

+------+----+-----+-------+--------+------+--------------------+--------------------+
|   Sex| Age|Sibsp|   Fare|Survived|Gender|          v_features|            features|
+------+----+-----+-------+--------+------+--------------------+--------------------+
|  male|22.0|    1|   7.25|     0.0|   0.0| [0.0,22.0,1.0,7.25]|[0.0,1.5144738264...|
|female|38.0|    1|71.2833|     1.0|   1.0|[1.0,38.0,1.0,71....|[2.07503065065606...|
|female|26.0|    0|  7.925|     1.0|   1.0|[1.0,26.0,0.0,7.925]|[2.07503065065606...|
|female|35.0|    1|   53.1|     1.0|   1.0| [1.0,35.0,1.0,53.1]|[2.07503065065606...|
|  male|35.0|    0|   8.05|     0.0|   0.0| [0.0,35.0,0.0,8.05]|[0.0,2.4093901784...|
|  male|54.0|    0|51.8625|     0.0|   0.0|[0.0,54.0,0.0,51....|[0.0,3.7173448467...|
|  male| 2.0|    3| 21.075|     0.0|   0.0|[0.0,2.0,3.0,21.075]|[0.0,0.1376794387...|
|female|27.0|    0|11.1333|     1.0|   1.0|[1.0,27.0,0.0,11....|[2.07503065065606...|
|female|14.0|    1|30.0708|     1.0|   1.0|[1.0,14.0,1

In [10]:
train,test = data.randomSplit([0.7, 0.3], seed=42)
log_reg = LogisticRegression(featuresCol='features', labelCol='Survived')
log_regModel = log_reg.fit(train)

In [11]:
predictions = log_regModel.transform(test)
predictions.show()

+------+----+-----+-------+--------+------+--------------------+--------------------+--------------------+--------------------+----------+
|   Sex| Age|Sibsp|   Fare|Survived|Gender|          v_features|            features|       rawPrediction|         probability|prediction|
+------+----+-----+-------+--------+------+--------------------+--------------------+--------------------+--------------------+----------+
|female| 1.0|    0|15.7417|     1.0|   1.0|[1.0,1.0,0.0,15.7...|[2.07503065065606...|[-1.7330699669381...|[0.15019531840662...|       1.0|
|female| 2.0|    1|   26.0|     1.0|   1.0|  [1.0,2.0,1.0,26.0]|[2.07503065065606...|[-1.4708363464906...|[0.18681552670407...|       1.0|
|female| 2.0|    3|   27.9|     0.0|   1.0|  [1.0,2.0,3.0,27.9]|[2.07503065065606...|[-0.7486381991553...|[0.32111810278665...|       1.0|
|female| 2.0|    4| 31.275|     0.0|   1.0|[1.0,2.0,4.0,31.275]|[2.07503065065606...|[-0.4195137832555...|[0.39663310338384...|       1.0|
|female| 4.0|    0| 22.025|

In [12]:
# evaluator = MulticlassClassificationEvaluator(labelCol="Survived",predictionCol="prediction")
# # bi_evaluator = BinaryClassificationEvaluator(labelCol="Survived",predictionCol="prediction")
# print('-----------------------------------')
# print('LR Train Metrics:')

# accuracy = evaluator.evaluate(predictions.select("Survived", "prediction"), {evaluator.metricName: "accuracy"})
# print("Accuracy : " + str(round(accuracy,2)))

# f1_Score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
# print("f1_Score : " + str(f1_Score))

In [13]:
metrics_multi = MulticlassMetrics(predictions.select("Survived", "prediction").rdd.map(tuple))

In [14]:
print('-----------------------------------')
print('LR Train Metrics:')

print("Accuracy : " + str(round(metrics_multi.accuracy,2)))
print("F1_Score : " + str(round(metrics_multi.fMeasure(1.0),2)))
print("Precision_Score : " + str(round(metrics_multi.precision(1.0),2)))
print("Recall_Score : " + str(round(metrics_multi.recall(1.0),2)))

-----------------------------------
LR Train Metrics:
Accuracy : 0.79
F1_Score : 0.72
Precision_Score : 0.68
Recall_Score : 0.76


In [21]:
import pandas as pd
pd.DataFrame(metrics_multi.confusionMatrix().toArray())


,0,1
0,97.0,24.0
1,16.0,51.0
